# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

ames = pd.read_csv('ames.csv',index_col=None,encoding='latin-1')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - np.mean(feature)) / np.std(feature)

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']
print('X:',np.size(X))
print('y:',np.size(y))

X: 68620
y: 1460


### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
# A brief preview of train-test split
print('X_train:', np.size(X_train), 'X_test:', np.size(X_test), 'y_train:', np.size(y_train), 'y_test:', np.size(y_test))
print('X_test/X_train:',np.divide(np.size(X_test),np.size(X_train)),'\ty_test/y_train:',np.divide(np.size(y_test),np.size(y_train)))

X_train: 54896 X_test: 13724 y_train: 1168 y_test: 292
X_test/X_train: 0.25 	y_test/y_train: 0.25


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
# Your code here
# Importing and initialize the linear regression model class
linreg = LinearRegression()
# Fit the model to train data
linreg.fit(X_train, y_train)
# Calculate predictions on training and test sets
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [6]:
# Your code here
# Calculate residuals
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test
# Import mean_squared_error from sklearn.metrics
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)
# Calculate training and test MSE
mse_train = np.dot((y_train-y_hat_train).T, y_train-y_hat_train) / np.size(y_train)
mse_test = np.dot((y_test-y_hat_test).T, y_test-y_hat_test) / np.size(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

Train Mean Squarred Error: 0.16036679992047
Test Mean Squarred Error: 0.17607390954119273
Train Mean Squarred Error: 0.16036679992047
Test Mean Squarred Error: 0.17607390954119276


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [7]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    df = pd.DataFrame(data)
    no = df.shape[0]
    fold_size = no // k
    left_over = no % k
    
    fold_lst = []
    count_os = 0
    
    for i in range(1,k+1):
        if i < left_over:
            # add 1 to fold size to account for leftovers
            fold = df.iloc[count_os:count_os+fold_size+1]
            fold_lst.append(fold)
            count_os += (fold_size + 1)
        else:
            fold = df.iloc[count_os:count_os+fold_size]
            fold_lst.append(fold)
            count_os += fold_size
    
    return fold_lst

### Apply it to the Ames Housing data

In [8]:
# Make sure to concatenate the data again
ames_data_check = pd.concat([X,y], axis=1)
ames_data_check.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
0,-0.133231,-0.803570,0.529260,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.560068
1,0.113442,0.418585,-0.381846,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.212764
2,0.420061,-0.576560,0.659675,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.734046
3,0.103347,-0.439287,0.541511,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.437382
4,0.878409,0.112267,1.282191,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.014651


In [9]:
# Make sure to concatenate the data again
ames_data = pd.concat([X.reset_index(drop=True) ,y], axis=1)
ames_data.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
0,-0.133231,-0.803570,0.529260,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.560068
1,0.113442,0.418585,-0.381846,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.212764
2,0.420061,-0.576560,0.659675,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.734046
3,0.103347,-0.439287,0.541511,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.437382
4,0.878409,0.112267,1.282191,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.014651


In [10]:
ames_data_check.shape == ames_data.shape

True

In [11]:
check_ok = ames_data_check.describe() == ames_data.describe()
print(list(check_ok.nunique()))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [12]:
lst = []
for col in check_ok.columns:
    lst.append((col,check_ok[col].unique().ravel()))
lst

[('LotArea_log', array([ True])),
 ('1stFlrSF_log', array([ True])),
 ('GrLivArea_log', array([ True])),
 ('BldgType_2fmCon', array([ True])),
 ('BldgType_Duplex', array([ True])),
 ('BldgType_Twnhs', array([ True])),
 ('BldgType_TwnhsE', array([ True])),
 ('KitchenQual_Fa', array([ True])),
 ('KitchenQual_Gd', array([ True])),
 ('KitchenQual_TA', array([ True])),
 ('SaleType_CWD', array([ True])),
 ('SaleType_Con', array([ True])),
 ('SaleType_ConLD', array([ True])),
 ('SaleType_ConLI', array([ True])),
 ('SaleType_ConLw', array([ True])),
 ('SaleType_New', array([ True])),
 ('SaleType_Oth', array([ True])),
 ('SaleType_WD', array([ True])),
 ('MSZoning_FV', array([ True])),
 ('MSZoning_RH', array([ True])),
 ('MSZoning_RL', array([ True])),
 ('MSZoning_RM', array([ True])),
 ('Street_Pave', array([ True])),
 ('Neighborhood_Blueste', array([ True])),
 ('Neighborhood_BrDale', array([ True])),
 ('Neighborhood_BrkSide', array([ True])),
 ('Neighborhood_ClearCr', array([ True])),
 ('Neig

In [13]:
# Apply kfolds() to ames_data with 5 folds
ames_folds = kfolds(ames_data, 5)

In [14]:
len(ames_folds)

5

In [15]:
ames_folds[0]

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
0,-0.133231,-0.803570,0.529260,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.560068
1,0.113442,0.418585,-0.381846,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.212764
2,0.420061,-0.576560,0.659675,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.734046
3,0.103347,-0.439287,0.541511,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.437382
4,0.878409,0.112267,1.282191,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.014651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,-0.209053,-0.796223,-1.539036,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1.600137
288,0.157048,-0.645759,-1.395708,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,-0.782026
289,-0.070210,-1.446006,-0.079200,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-0.205619
290,1.053400,-0.074653,0.875085,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.840763


### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [16]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(ames_folds) if i != n])
    test = ames_folds[n]
    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']
    X_test = test.drop('SalePrice_log', axis=1)
    y_test = test['SalePrice_log']
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    # Evaluate Train and Test errors
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    train_errs.append(np.mean(train_residuals.astype('float64')**2))
    test_errs.append(np.mean(test_residuals.astype('float64')**2))
    
print(train_errs)
print(test_errs)

[0.17182278335255932, 0.15518564839930488, 0.15670679668461976, 0.16145616307615304, 0.1517544269196179]
[0.1244006674209639, 0.19363327184178997, 0.18923491727014616, 0.17091031435941947, 0.2075692165854609]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [17]:
# Your code here
cv_5_results  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_10_results = np.mean(cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error'))
cv_20_results = np.mean(cross_val_score(linreg, X, y, cv=20, scoring='neg_mean_squared_error'))

In [18]:
mse = make_scorer(mean_squared_error)

cv_5_results_2  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring=mse))
cv_10_results_2 = np.mean(cross_val_score(linreg, X, y, cv=10, scoring=mse))
cv_20_results_2 = np.mean(cross_val_score(linreg, X, y, cv=20, scoring=mse))

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [19]:
# Your code here
print(cv_5_results.mean(),cv_10_results.mean(),cv_20_results.mean())
print(cv_5_results_2.mean(),cv_10_results_2.mean(),cv_20_results_2.mean())

-0.17714967749555607 -0.17595544685191153 -0.17664320259283411
0.17714967749555607 0.17595544685191153 0.17664320259283411


##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 